### 1. 라이브러리 로드

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os

from google.colab import drive
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# 시각화 설정
sns.set_theme(style="whitegrid")
plt.rcParams['font.family'] = 'sans-serif'

drive.mount('/content/drive')

base_path = '/content/drive/MyDrive/f1_project/data/'
model_path = os.path.join(base_path, 'f1_excitement_model.pkl')
scaler_path = os.path.join(base_path, 'scaler_features.pkl')

print(f"Loading Model from: {model_path}")

test_file_name = 'finaltest_data.csv'
test_file_path = os.path.join(base_path, test_file_name)
print(f"Loading Test Data from: {test_file_path}")

# 2. 모델 및 스케일러 로드
try:
    model = joblib.load(model_path)
    scaler = joblib.load(scaler_path)
    print("✅ Model & Scaler loaded successfully!")
    print(f"   - Model Type: {type(model).__name__}")
    print(f"   - N Estimators: {model.n_estimators}")
except FileNotFoundError:
    print("❌ 모델 파일을 찾을 수 없습니다. training.ipynb를 먼저 실행하여 모델을 저장해주세요.")
    raise

### 2. 가상 시나리오 생성

In [ ]:
# -----------------------------------------------------------------------------
# 3. 가상 시나리오 생성 (Hypothetical Scenarios)
# 모델이 학습한 4가지 Feature에 대해 가상의 값을 입력합니다.
# Features: ['Chaos_Score', 'Lead_Changes', 'Gap_Std_Dev', 'Position_Gains_Total']
# -----------------------------------------------------------------------------

# 시나리오 A: "수면제 경기" (Sleep-inducing Race)
# - 사고 없음(0), 선두 교체 없음(0), 1-2위 격차 큼(25초), 추월 거의 없음(5회)
scenario_boring = [0, 0, 25.0, 5]

# 시나리오 B: "평범한 중위권 싸움" (Average Midfield Battle)
# - 사고 조금(2), 선두 교체 1회, 격차 적당함(8초), 추월 적당히 있음(30회)
scenario_average = [2, 1, 8.0, 30]

# 시나리오 C: "역대급 대혼돈의 경기" (Chaos & Thriller)
# - 사고 다수(8), 선두 교체 5회, 초박빙 격차(0.5초), 추월 엄청 많음(80회)
scenario_chaos = [8, 5, 0.5, 80]

# 데이터프레임으로 변환
features = ['Chaos_Score', 'Lead_Changes', 'Gap_Std_Dev', 'Position_Gains_Total']
scenarios_df = pd.DataFrame([scenario_boring, scenario_average, scenario_chaos],
                            columns=features,
                            index=['Scenario A (Boring)', 'Scenario B (Average)', 'Scenario C (Chaos)'])

print("\n[Input Scenarios]")
display(scenarios_df)

# -----------------------------------------------------------------------------
# 4. 추론 실행 (Inference)
# -----------------------------------------------------------------------------

# 스케일링 (학습 때 사용한 Scaler로 transform)
X_input_scaled = scaler.transform(scenarios_df)

# 예측 (0.0 ~ 1.0 사이 값 반환)
predictions = model.predict(X_input_scaled)

### 3. 시각화

In [ ]:
# -----------------------------------------------------------------------------
# 5. 결과 해석 및 시각화 (Interpretation)
# -----------------------------------------------------------------------------

print("\n" + "="*40)
print("🤖 AI Excitement Prediction Results")
print("="*40)

results = []
for name, score in zip(scenarios_df.index, predictions):
    # 0~1 스케일을 10점 만점으로 변환하여 출력
    score_10 = score * 10.0
    results.append({'Scenario': name, 'Predicted_Score': score_10})

    # 텍스트로 결과 해석 출력
    rating = "⭐⭐⭐⭐⭐" if score_10 >= 8 else ("⭐⭐⭐" if score_10 >= 5 else "⭐")
    print(f"👉 {name:20}: {score_10:.2f}점 / 10.0 {rating}")

print("="*40)

# 시각화 (Bar Chart)
results_df = pd.DataFrame(results)
plt.figure(figsize=(10, 5))
colors = ['gray', 'skyblue', 'red'] # 노잼, 평범, 꿀잼 색상 다르게

sns.barplot(data=results_df, x='Scenario', y='Predicted_Score', hue='Scenario', palette=colors, legend=False)
plt.ylim(0, 10) # 10점 만점 고정
plt.axhline(y=5, color='black', linestyle='--', alpha=0.3, label='Average Threshold')
plt.ylabel('Excitement Score (0-10)')
plt.title('AI Prediction for Hypothetical Scenarios')

# 막대 위에 점수 표시
for index, row in results_df.iterrows():
    plt.text(index, row.Predicted_Score + 0.2, f"{row.Predicted_Score:.1f}", ha='center', fontweight='bold')

plt.show()

## Summary

### 1. 추론 시나리오 설계 (Hypothetical Scenario Design)
- 실제 데이터 없이도 모델의 예측 논리를 검증하기 위해, F1 경기의 특징을 반영한 **3가지 가상 시나리오**를 설계했습니다.
    - **Scenario A** (Boring): 사고(0), 추월(5), 큰 격차(25s) → "노잼 경기"
    - **Scenario B** (Average): 일반적인 사고(2)와 추월(30) 빈도 → "평범한 중위권 싸움"
    - **Scenario C** (Chaos): 다수의 사고(8), 잦은 선두 교체(5), 초박빙(0.5s) → "상당히 혼란스러운 경기"

### 2. 프로세스의 재현 가능성 (Reproducibility)
- `training.ipynb`에서 학습되고 저장된 모델(`f1_excitement_model.pkl`)과 스케일러(`scaler_features.pkl`)를 로드하여 사용했습니다.
- 학습 단계와 동일한 Feature 순서 및 전처리(Scaling) 방식을 적용하여, 일관성 있는 추론 결과를 보장했습니다.

### 3. 결과 해석 및 시각화 (Interpretation & Visualization)
- 모델이 예측한 정규화된 값(0~1)을 **10점 만점 스케일**로 변환하여 직관적인 이해를 도왔습니다.
- 점수에 따라 **별점**을 부여하여 비전문가도 결과를 쉽게 해석할 수 있도록 구성했습니다.
- 시나리오별 예측 점수를 색상(Gray/Skyblue/Red)으로 구분한 막대그래프로 시각화하여, 모델이 "입력 변수(사고, 추월 등)가 증가할수록 흥행 점수를 높게 예측한다"는 논리적 타당성을 입증했습니다.